In [1]:
from transformers import pipeline
unmasker = pipeline('fill-mask', model='roberta-base')
unmasker("Hello I'm a <mask> model.")

/home/or/.virtualenvs/biu_task2/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Downloading: 100%|███████████████████████████| 501M/501M [02:12<00:00, 3.79MB/s]
Downloading: 100%|████████████████████████████| 899k/899k [00:01<00:00, 847kB/s]
Downloading: 100%|████████████████████████████| 456k/456k [00:00<00:00, 535kB/s]
Downloading: 100%|█████████████████████████| 1.36M/1.36M [00:01<00:00, 1.21MB/s]


[{'score': 0.3306529223918915,
  'token': 2943,
  'token_str': ' male',
  'sequence': "Hello I'm a male model."},
 {'score': 0.04655393213033676,
  'token': 2182,
  'token_str': ' female',
  'sequence': "Hello I'm a female model."},
 {'score': 0.04233003035187721,
  'token': 2038,
  'token_str': ' professional',
  'sequence': "Hello I'm a professional model."},
 {'score': 0.03721676394343376,
  'token': 2734,
  'token_str': ' fashion',
  'sequence': "Hello I'm a fashion model."},
 {'score': 0.03253666311502457,
  'token': 1083,
  'token_str': ' Russian',
  'sequence': "Hello I'm a Russian model."}]

In [5]:
text = "I am so <mask>"

In [43]:
import torch
import torch.nn.functional as F
import numpy as np
from transformers import RobertaTokenizer, RobertaForMaskedLM

model_checkpoint = 'roberta-base'

model = RobertaForMaskedLM.from_pretrained(model_checkpoint)
tokenizer = RobertaTokenizer.from_pretrained(model_checkpoint)

inputs = tokenizer(text, return_tensors="pt")

for code in inputs['input_ids'][0]:
    print(f"{ tokenizer.decode(code)} : {code}")
    
with torch.no_grad():
    logits = model(**inputs).logits
    
am_loc = 2
mask_loc = 4
def k_most_similar(logits, index):
    mask_token_logits = logits[0, index, :]
    # Pick the [MASK] candidates with the highest logits
    probabilities = F.softmax(mask_token_logits,dim=0)
    top_5_tokens = np.argsort(-probabilities)[:5].tolist()
    
    for token in top_5_tokens:
        print(f">>> {text.replace(tokenizer.mask_token, tokenizer.decode([token]))},    (probability:{probabilities[token]})")

print('\n------------------------\n')
print("5 most similar to 'am':") 
k_most_similar(logits, am_loc)
print('\n------------------------')
print("5 most similar to '<mask>':") 
k_most_similar(logits, mask_loc)

<s> : 0
I : 100
 am : 524
 so : 98
<mask> : 50264
</s> : 2

------------------------

5 most similar to 'am':
>>> I am so  am,    (probability:0.9998922348022461)
>>> I am so  is,    (probability:3.9378628571284935e-05)
>>> I am so 'm,    (probability:2.9937518775113858e-05)
>>> I am so  was,    (probability:8.688964953762479e-06)
>>> I am so  feel,    (probability:8.550764505343977e-06)

------------------------
5 most similar to '<mask>':
>>> I am so  sorry,    (probability:0.3083705008029938)
>>> I am so  proud,    (probability:0.0649036392569542)
>>> I am so  grateful,    (probability:0.05806168541312218)
>>> I am so  happy,    (probability:0.04478686675429344)
>>> I am so  blessed,    (probability:0.032352522015571594)


In [39]:
F.softmax(mask_token_logits,dim=0)

tensor([8.0206e-14, 1.6637e-15, 5.3132e-09,  ..., 6.7266e-14, 3.4216e-14,
        2.4555e-12])

In [34]:
tokenizer.encode(['good'])

[0, 8396, 2]

In [31]:
mask_token_logits = logits[0, 2, :]

In [33]:
mask_token_logits.shape

torch.Size([50265])